In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from cellpy.utils import easyplot


from cellpy import prms
from cellpy.utils.batch_tools.batch_journals import LabJournal

In [ ]:
f1 = Path("data/raw/20160805_test001_45_cc_01.res")
f2 = Path("data/raw/20160805_test001_45_cc_02.res")

In [ ]:
f1.is_file()

In [ ]:
f2.is_file()

# Checking standard usage

In [ ]:
files = [f1, f2]
names = [f1.name, f2.name]

In [ ]:
ezplt = easyplot.EasyPlot(files, names, figtitle="Test1")

In [ ]:
ezplt.plot()

### NOTE
- easyplot crashes if nicknames are not given

In [ ]:
easyplot.EasyPlot(
    files,
    names,
    figtitle="Test2",
    galvanostatic_normalize_capacity=True,
    all_in_one=True,
    dqdv_plot=True,
).plot()

In [ ]:
f1.with_name(f"{f1.stem}_tmp.xlsx")

In [ ]:
easyplot.help()

## Loading journal

In [3]:
journal_file = Path("../../dev_data/db/test_journal.xlsx")

In [4]:
journal_file.is_file()

True

In [5]:
j = LabJournal(db_reader=None)

In [6]:
j.from_file(journal_file, paginate=False)

CRITICAL:root:no session found in your journal file


In [7]:
j.pages

,keep,id_key,mass,total_mass,loading,nom_cap,experiment,cell_type,instrument,comment,fixed,label,cellpy_file_name,raw_file_names,group,sub_group
filename,,,,,,,,,,,,,,,,
test001,1.0,1.0,0.510262,2.8175,0.956369,2970.0,cycling,anode,arbin_res,test,0.0,cell001,data\cellpyfiles\20160805_test001_45_cc_01.cellpy,data/raw/20160805_test001_45_cc_01.res,1.0,1.0
test002,1.0,2.0,0.454388,2.5375,0.861326,2926.0,cycling,anode,arbin_res,test,0.0,cell002,data\cellpyfiles\20160805_test001_45_cc_02.cellpy,data/raw/20160805_test001_45_cc_02.res,1.0,2.0
test003,1.0,3.0,0.495843,2.6975,0.915636,3454.0,cycling,anode,arbin_res,test,0.0,cell003,data\cellpyfiles\20160805_test001_45_cc_03.cellpy,data/raw/20160805_test001_45_cc_03.res,1.0,3.0
test004,1.0,4.0,0.454388,2.8975,0.983475,3080.0,cycling,anode,arbin_res,test,0.0,cell004,data\cellpyfiles\20160805_test001_45_cc_04.cellpy,data/raw/20160805_test001_45_cc_04.res,1.0,4.0
test005,1.0,5.0,0.447178,2.7875,0.946138,3117.4,cycling,anode,arbin_res,test,0.0,cell005,data\cellpyfiles\20160805_test001_45_cc_05.cellpy,data/raw/20160805_test001_45_cc_05.res,2.0,1.0


### NOTE
- should rename index to cell_name instead of filename

In [ ]:
rawfiles = j.pages.raw_file_names.to_list()

In [ ]:
names = j.pages.label.to_list()

In [ ]:
masses = j.pages.mass.to_list()

In [ ]:
easyplot.EasyPlot(
    rawfiles,
    names,
    figtitle="Test3",
    galvanostatic_normalize_capacity=True,
    all_in_one=True,
    dqdv_plot=True,
).plot()

In [ ]:
outfile = journal_file

In [ ]:
j.to_file(outfile, to_project_folder=False)

### Note

- need to implement a way to export journals to xlsx
- maybe we also need to allow for reading meta and session from xlsx journals
- figure out or modify easyplot so that mass and other parameters can be given to it
- figure out a way to save an easyplot session with session and metadata

## Save journal to xlsx

In [ ]:
bad_cycle_numbers = {"cell_name": ['20160805_test001_45_cc', '20160805_test001_45_cc', '20160805_test001_45_cc', '20160805_test001_47_cc', '20160805_test001_47_cc']
        '20160805_test001_45_cc': [4, 337, 338],
        '20160805_test001_47_cc': [7, 8, 9, 33],
}

In [142]:
bad_cycle_numbers = {
    "20160805_test001_45_cc": [4, 337, 338],
    "20160805_test001_47_cc": [7, 8, 9, 33],
}
bad_cells = ["20160805_test001_45_cc", "another_cell_000_cc"]

notes = ["one comment for the road", "another comment", "a third comment"]

session0 = {
    "bad_cycle_numbers": bad_cycle_numbers,
    "bad_cells": bad_cells,
    "notes": notes,
}

meta = j._prm_packer()

In [73]:
meta

{'name': 'test_journal',
 'project': 'NaN',
 'time_stamp': None,
 'project_dir': WindowsPath('.'),
 'batch_dir': WindowsPath('.'),
 'raw_dir': WindowsPath('.')}

In [74]:
session

{'bad_cycle_numbers': {'20160805_test001_45_cc': [4, 337, 338],
  '20160805_test001_47_cc': [7, 8, 9, 33]},
 'bad_cells': ['20160805_test001_45_cc', 'another_cell_000_cc'],
 'notes': ['one comment for the road', 'another comment', 'a third comment']}

In [75]:
import pandas as pd

In [90]:
l_bad_cycle_numbers = []
for k, v in bad_cycle_numbers.items():
    l_bad_cycle_numbers.append(pd.DataFrame(data=v, columns=[k]))

df_bad_cycle_numbers = (
    pd.concat(l_bad_cycle_numbers, axis=1)
    .melt(var_name="cell_name", value_name="cycle_index")
    .dropna()
)

In [91]:
df_bad_cycle_numbers

,cell_name,cycle_index
0,20160805_test001_45_cc,4.0
1,20160805_test001_45_cc,337.0
2,20160805_test001_45_cc,338.0
4,20160805_test001_47_cc,7.0
5,20160805_test001_47_cc,8.0
6,20160805_test001_47_cc,9.0
7,20160805_test001_47_cc,33.0


In [92]:
df_bad_cells = pd.DataFrame(bad_cells, columns=["cell_name"])
df_bad_cells

,cell_name
0,20160805_test001_45_cc
1,another_cell_000_cc


In [93]:
df_notes = pd.DataFrame(notes, columns=["txt"])
df_notes

,txt
0,one comment for the road
1,another comment
2,a third comment


In [98]:
len(meta)

6

In [108]:
df_meta = pd.DataFrame(meta, index=[0]).melt(var_name="parameter", value_name="value")

In [109]:
df_meta

,parameter,value
0,name,test_journal
1,project,NaN
2,time_stamp,None
3,project_dir,.
4,batch_dir,.
5,raw_dir,.


In [94]:
session = pd.concat(
    [df_bad_cycle_numbers, df_bad_cells, df_notes],
    axis=1,
    keys=["bad_cycle_number", "bad_cells", "notes"],
)

In [95]:
session

bad_cycle_number                           bad_cells  \
                cell_name cycle_index               cell_name   
0  20160805_test001_45_cc         4.0  20160805_test001_45_cc   
1  20160805_test001_45_cc       337.0     another_cell_000_cc   
2  20160805_test001_45_cc       338.0                     NaN   
4  20160805_test001_47_cc         7.0                     NaN   
5  20160805_test001_47_cc         8.0                     NaN   
6  20160805_test001_47_cc         9.0                     NaN   
7  20160805_test001_47_cc        33.0                     NaN   

                      notes  
                        txt  
0  one comment for the road  
1           another comment  
2           a third comment  
4                       NaN  
5                       NaN  
6                       NaN  
7                       NaN

In [65]:
file_name = Path("out.xlsx")

In [96]:
pages = j.pages

In [225]:
# saving journal to xlsx
try:
    with pd.ExcelWriter(file_name, mode="w", engine="openpyxl") as writer:
        pages.to_excel(writer, sheet_name="pages", engine="openpyxl")
        # no index is not supported for multi-index (update to index=False when pandas implements it):
        session.to_excel(writer, sheet_name="session", engine="openpyxl")
        df_meta.to_excel(writer, sheet_name="meta", engine="openpyxl", index=False)
except PermissionError as e:
    print(f"Could not load journal to xlsx ({e})")

## loading xlsx journal

In [226]:
import tempfile
import shutil

In [227]:
# loading journal from xlsx: pages
temporary_directory = tempfile.mkdtemp()
temporary_file_name = shutil.copy(file_name, temporary_directory)
try:
    pages2 = pd.read_excel(temporary_file_name, sheet_name="pages", engine="openpyxl")
except PermissionError as e:
    print(f"Could not load journal to xlsx ({e})")

pages2

,filename,keep,id_key,mass,total_mass,loading,nom_cap,experiment,cell_type,instrument,comment,fixed,label,cellpy_file_name,raw_file_names,group,sub_group
0,test001,1,1,0.510262,2.8175,0.956369,2970.0,cycling,anode,arbin_res,test,0,cell001,data\cellpyfiles\20160805_test001_45_cc_01.cellpy,data/raw/20160805_test001_45_cc_01.res,1,1
1,test002,1,2,0.454388,2.5375,0.861326,2926.0,cycling,anode,arbin_res,test,0,cell002,data\cellpyfiles\20160805_test001_45_cc_02.cellpy,data/raw/20160805_test001_45_cc_02.res,1,2
2,test003,1,3,0.495843,2.6975,0.915636,3454.0,cycling,anode,arbin_res,test,0,cell003,data\cellpyfiles\20160805_test001_45_cc_03.cellpy,data/raw/20160805_test001_45_cc_03.res,1,3
3,test004,1,4,0.454388,2.8975,0.983475,3080.0,cycling,anode,arbin_res,test,0,cell004,data\cellpyfiles\20160805_test001_45_cc_04.cellpy,data/raw/20160805_test001_45_cc_04.res,1,4
4,test005,1,5,0.447178,2.7875,0.946138,3117.4,cycling,anode,arbin_res,test,0,cell005,data\cellpyfiles\20160805_test001_45_cc_05.cellpy,data/raw/20160805_test001_45_cc_05.res,2,1


In [228]:
# loading journal from xlsx: session
temporary_directory = tempfile.mkdtemp()
temporary_file_name = shutil.copy(file_name, temporary_directory)
try:
    session2 = pd.read_excel(
        temporary_file_name, sheet_name="session", engine="openpyxl", header=[0, 1]
    )
except PermissionError as e:
    print(f"Could not load journal to xlsx ({e})")
session2

Unnamed: 0_level_0        bad_cycle_number              \
  Unnamed: 0_level_1               cell_name cycle_index   
0                  0  20160805_test001_45_cc           4   
1                  1  20160805_test001_45_cc         337   
2                  2  20160805_test001_45_cc         338   
3                  4  20160805_test001_47_cc           7   
4                  5  20160805_test001_47_cc           8   
5                  6  20160805_test001_47_cc           9   
6                  7  20160805_test001_47_cc          33   

                bad_cells                     notes  
                cell_name                       txt  
0  20160805_test001_45_cc  one comment for the road  
1     another_cell_000_cc           another comment  
2                     NaN           a third comment  
3                     NaN                       NaN  
4                     NaN                       NaN  
5                     NaN                       NaN  
6                     NaN                       NaN

In [229]:
bcn2 = {
    l: list(sb["cycle_index"].values)
    for l, sb in session2["bad_cycle_number"].groupby("cell_name")
}
bcn2

{'20160805_test001_45_cc': [4, 337, 338],
 '20160805_test001_47_cc': [7, 8, 9, 33]}

In [230]:
bc2 = list(session2["bad_cells"].dropna().values.flatten())
bc2

['20160805_test001_45_cc', 'another_cell_000_cc']

In [231]:
n2 = list(session2["notes"].dropna().values.flatten())
n2

['one comment for the road', 'another comment', 'a third comment']

In [232]:
session3 = {"bad_cycle_numbers": bcn2, "bad_cells": bc2, "notes": n2}

In [233]:
session0

{'bad_cycle_numbers': {'20160805_test001_45_cc': [4, 337, 338],
  '20160805_test001_47_cc': [7, 8, 9, 33]},
 'bad_cells': ['20160805_test001_45_cc', 'another_cell_000_cc'],
 'notes': ['one comment for the road', 'another comment', 'a third comment']}

In [234]:
session3

{'bad_cycle_numbers': {'20160805_test001_45_cc': [4, 337, 338],
  '20160805_test001_47_cc': [7, 8, 9, 33]},
 'bad_cells': ['20160805_test001_45_cc', 'another_cell_000_cc'],
 'notes': ['one comment for the road', 'another comment', 'a third comment']}

In [235]:
session0 == session3

True

In [236]:
# loading journal from xlsx: meta
temporary_directory = tempfile.mkdtemp()
temporary_file_name = shutil.copy(file_name, temporary_directory)
try:
    df_meta2 = pd.read_excel(
        temporary_file_name, sheet_name="meta", engine="openpyxl", index_col="parameter"
    )
except PermissionError as e:
    print(f"Could not load journal to xlsx ({e})")

In [237]:
meta2 = df_meta2.to_dict()["value"]
meta2

{'name': 'test_journal',
 'project': nan,
 'time_stamp': nan,
 'project_dir': '.',
 'batch_dir': '.',
 'raw_dir': '.'}

In [238]:
meta

{'name': 'test_journal',
 'project': 'NaN',
 'time_stamp': None,
 'project_dir': WindowsPath('.'),
 'batch_dir': WindowsPath('.'),
 'raw_dir': WindowsPath('.')}

In [239]:
pages2

,filename,keep,id_key,mass,total_mass,loading,nom_cap,experiment,cell_type,instrument,comment,fixed,label,cellpy_file_name,raw_file_names,group,sub_group
0,test001,1,1,0.510262,2.8175,0.956369,2970.0,cycling,anode,arbin_res,test,0,cell001,data\cellpyfiles\20160805_test001_45_cc_01.cellpy,data/raw/20160805_test001_45_cc_01.res,1,1
1,test002,1,2,0.454388,2.5375,0.861326,2926.0,cycling,anode,arbin_res,test,0,cell002,data\cellpyfiles\20160805_test001_45_cc_02.cellpy,data/raw/20160805_test001_45_cc_02.res,1,2
2,test003,1,3,0.495843,2.6975,0.915636,3454.0,cycling,anode,arbin_res,test,0,cell003,data\cellpyfiles\20160805_test001_45_cc_03.cellpy,data/raw/20160805_test001_45_cc_03.res,1,3
3,test004,1,4,0.454388,2.8975,0.983475,3080.0,cycling,anode,arbin_res,test,0,cell004,data\cellpyfiles\20160805_test001_45_cc_04.cellpy,data/raw/20160805_test001_45_cc_04.res,1,4
4,test005,1,5,0.447178,2.7875,0.946138,3117.4,cycling,anode,arbin_res,test,0,cell005,data\cellpyfiles\20160805_test001_45_cc_05.cellpy,data/raw/20160805_test001_45_cc_05.res,2,1


In [134]:
session2

,Unnamed: 0,bad_cycle_number,Unnamed: 2,bad_cells,notes
0,NaN,cell_name,cycle_index,cell_name,txt
1,NaN,NaN,NaN,NaN,NaN
2,0.0,20160805_test001_45_cc,4,20160805_test001_45_cc,one comment for the road
3,1.0,20160805_test001_45_cc,337,another_cell_000_cc,another comment
4,2.0,20160805_test001_45_cc,338,NaN,a third comment
5,4.0,20160805_test001_47_cc,7,NaN,NaN
6,5.0,20160805_test001_47_cc,8,NaN,NaN
7,6.0,20160805_test001_47_cc,9,NaN,NaN
8,7.0,20160805_test001_47_cc,33,NaN,NaN


In [135]:
df_meta2

,Unnamed: 0,parameter,value
0,0,name,test_journal
1,1,project,NaN
2,2,time_stamp,NaN
3,3,project_dir,.
4,4,batch_dir,.
5,5,raw_dir,.


In [57]:
j._prm_packer()

{'name': 'test_journal',
 'project': 'NaN',
 'time_stamp': None,
 'project_dir': WindowsPath('.'),
 'batch_dir': WindowsPath('.'),
 'raw_dir': WindowsPath('.')}